# Abstract
Huffman Coding is an algorithm used for lossless data compression. In this article we would describe and implement the algorithm. Finally, we will present a demonstration of its compression rate and how it compares to other compression algorithms.

#### Keywords
Huffman Coding, Huffman tree, lossless compression, entropy

# Introduction
The Huffman Coding is a lossless data compression algorithm, developed by David Huffman in the early of 50s while he was a PhD student at MIT. The algorithm is based on a binary-tree frequency-sorting method that allow encode any message sequence into shorter encoded messages and a method to reassemble into the original message without losing any data.

### Lossless vs Lossy compression
Lossless and lossy file compression describe whether all original data can be recovered when the file is uncompressed.

With lossless compression, every bit of data originally in a file remains after it is uncompressed, and all the information is restored. Lossy compression reduces a file by permanently eliminating certain information, especially redundant information.

When the file is uncompressed, some of the original information is not there, although the user may not notice it.

### Entropy
In information theory, Entropy is a measure of the "disorder"/randomness in a system. The entropy of a set of data is related to the amount of information that it contains which is directly related to the amount of compression that can be achieved. For example the more unique and randomly positioned letters we have in a sample text, the less compression we can achieve.

* `TTTTTTTTT` can be compressed to `T9`
* `ABCDEFGHI` string has the same length (9 characters), but higher entropy. Thus, lower possible compression rate.

Entropy is the measure of redundancy or randomness of data, including strings. Highly random data will have an even distribution of tokens and will contain few meaningful patterns and high entropy. English text is redundant because the appearance of a 'q' generally precedes 'u'. 't' following 's' is a good guess too. 

Basically entropy rate establishes the statistical limits to possible data compression for random data.
For example, the entropy rate of the English language is often cited as 1.5 bits per character (give or take). Typical encodings use 8 bits per character. So a maximally compressed text should be 1.5/8 (~19%) the size of the original

### Heap
A heap is a special type of tree-like structure that follows a specific rule. It as a collection of values arranged in a particular way. For example, in a "max" heap, every parent value is greater than or equal to its children. In a "min" heap, it’s the opposite.

![Heap structure](img/huffman/heap.png)

This arrangement allows us to easily find the highest or lowest value in the heap, which is located at the top.

In Huffman algorithm heap is being used for ordering letters based on their frequency in the text.

### Huffman trees
Huffman trees are binary trees (every node has only 2 children). Huffman tree basically sort the characters found in a text by their frequency. The more often a character occurs in the text, the higher it will be placed in the tree. However, all characters are leaf nodes. The interim nodes don't hold a character, but they hold the sum of the leafs below them.

![Huffman tree](img/huffman/huffman_tree.png)

Once we build the tree (based on char frequencies) we convert the data, piece-by-piece, into a binary code using the binary tree. In result, the higher frequency a character has, the shorter bit representation it gets. Latter serves best the compression interests.

##### Decoding a binary code into text
We need the hoffman tree (as a map) to decode the binary file. So, using that same binary tree we can simply start building up blocks of bits, until we find a code match (a character leaf node). There is no risk of conflict (e.g. two codes starting with the same sequence of bits), since all the characters are leaf nodes with unique bit representation *(look at the two examples in the figure above)*.

# Implementation
A simple Python implementation is provided below. Both compression (encoding) and decompression (decoding) is implemented.

In [1]:
from io import TextIOWrapper
import os
import heapq
    
class HeapNode:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq

    def __eq__(self, other):
        if other == None:
            return False
        if not isinstance(other, HeapNode):
            return False
        return self.freq == other.freq

class Huffman:

    input_file_path: str = ""
    frequency: dict[str, int] = {}
    heap: list[HeapNode] = []
    codes: dict[str, str] = {}

    def __init__(self, input_file_path: str):
        self.input_file_path = input_file_path

    def __calculate_frequencies(self, text: str):
        '''Count the frequency of each character in the text and return the generated frequency dictionary.'''
        self.frequency = {}
        for char in text:
            if not char in self.frequency:
                self.frequency[char] = 0
            self.frequency[char] += 1
        return self.frequency

    def __build_heap(self):
        '''Build a heap from the frequency dictionary where the priority is the frequency of the character.'''
        # Create a heap node for each character in the frequency dictionary. Go from the smallest frequency to the largest.
        for char in self.frequency:
            node = HeapNode(char, self.frequency[char])
            heapq.heappush(self.heap, node)

    def __merge_heap_nodes(self):
        '''Build a huffman tree and save its root node in the heap.'''
        # While there is more than one node in the heap, merge the two nodes with the smallest frequencies.
        while len(self.heap) > 1:
            node1 = heapq.heappop(self.heap) # Pop the node with the smallest frequency.
            node2 = heapq.heappop(self.heap) # Pop the node with the second smallest frequency.

            # Merged nodes are not characters, so we set the character to None.
            merged = HeapNode(None, node1.freq + node2.freq)
            merged.left = node1 # Left node is the smaller frequency node.
            merged.right = node2 # Right node is the larger frequency node.

            # Push the merged node back into the heap.
            heapq.heappush(self.heap, merged)

    def __traverse_tree_and_assign_codes(self):
        '''Assign codes to characters in the tree.'''
        root_node = heapq.heappop(self.heap) # Pop the root node from the heap.

        # Traverse the tree and assign codes to characters.
        code = ""
        self.__visit_nodes_and_assign_codes_recursively(root_node, code)

    def __visit_nodes_and_assign_codes_recursively(self, node: HeapNode, code: str):
        '''Assign codes to characters in the tree.'''
        if node is None:
            return

        # If the node is a character, assign the code to the character.
        if node.char is not None:
            self.codes[node.char] = code

        # Traverse the left sub-node and assign a 0 to its code.
        self.__visit_nodes_and_assign_codes_recursively(node.left, code + "0")
        # Traverse the right sub-node and assign a 1 to its code.
        self.__visit_nodes_and_assign_codes_recursively(node.right, code + "1")

    def __get_encoded_text(self, text: str) -> str:
        '''Replace characters with their corresponding codes.'''
        encoded_text = ""
        for char in text:
            encoded_text += self.codes[char]
        return encoded_text

    def __pad_encoded_text(self, encoded_text: str) -> str:
        '''Add padding (zeros) at the end of the encoded text to make it a multiple of 8.'''
        # Calculate the number of padding bits needed to make the encoded text a multiple of 8.
        extra_padding = 8 - len(encoded_text) % 8

        # Add padding bits to the end of the encoded text.
        for _ in range(extra_padding):
            encoded_text += "0"

        # Add the number of padding bits to the start of the encoded text.
        padded_info = "{0:08b}".format(extra_padding)
        padded_encoded_text = padded_info + encoded_text
        return padded_encoded_text

    def __save_output_file(self, padded_encoded_text: str, output_file: TextIOWrapper):
        data_bytes = self.__convert_to_bytes(padded_encoded_text)
        output_file.write(bytes(data_bytes))

    def __convert_to_bytes(self, padded_encoded_text: str) -> list:
        '''Convert the padded encoded text to a list of bytes.'''
        if len(padded_encoded_text) % 8 != 0:
            print("Encoded text not padded properly")
            exit(0)

        data_bytes = bytearray()
        for i in range(0, len(padded_encoded_text), 8):
            byte = padded_encoded_text[i:i+8] # Get the next 8 bits.
            data_bytes.append(int(byte, 2)) # Convert the 8 bits to an integer.
        return data_bytes
    
    def __remove_padding(self, padded_encoded_text: str) -> str:
        '''Remove padding (zeros) from the end of the encoded text.'''
        padded_info = padded_encoded_text[:8] # Get the number of padding bits.
        extra_padding = int(padded_info, 2) # Convert the number of padding bits to an integer.
        padded_encoded_text = padded_encoded_text[8:] # Remove the number of padding bits from the encoded text.
        encoded_text = padded_encoded_text[:-extra_padding] # Remove the padding bits from the end of the encoded text.
        return encoded_text
    
    def __decode_text(self, encoded_text: str) -> str:
        '''Decode the encoded text.'''
        current_code = ""
        decoded_text = ""
        for bit in encoded_text:
            current_code += bit
            if current_code in self.codes.values(): # If the current code is in the codes dictionary, add the character to the decoded text.
                key_index = list(self.codes.values()).index(current_code) # Get the key of the current code.
                decoded_text += list(self.codes.keys())[key_index] # Add the character to the decoded text.
                current_code = "" # Reset the current code.
        return decoded_text

    def compress(self):
        file_name, file_extension = os.path.splitext(self.input_file_path)
        output_file_path = file_name + ".bin"

        with open(self.input_file_path, 'r') as input_file, open(output_file_path, 'wb') as output_file:
            text = input_file.read()
            text = text.rstrip()

            self.__calculate_frequencies(text)
            self.__build_heap()
            self.__merge_heap_nodes()
            self.__traverse_tree_and_assign_codes()
            encoded_text = self.__get_encoded_text(text)
            padded_encoded_text = self.__pad_encoded_text(encoded_text)

            self.__save_output_file(padded_encoded_text, output_file)

        # Print the size of the original file and the size of the compressed file.
        print("The file was compressed successfully.")
        print(f"\tOriginal file size: {os.path.getsize(self.input_file_path)} bytes")
        print(f"\tCompressed file size: {os.path.getsize(output_file_path)} bytes")
        return output_file_path
        
    def decompress(self, input_file_path: str):
        file_name, file_extension = os.path.splitext(input_file_path)
        output_file_path = file_name + "_decompressed.txt"

        with open(input_file_path, 'rb') as input_file, open(output_file_path, 'w') as output_file:
            bit_string = ""

            byte = input_file.read(1)
            while byte and len(byte) > 0:
                byte = ord(byte) # Convert the byte to an integer.
                bits = bin(byte)[2:].rjust(8, '0') # Convert the integer to a binary string.
                bit_string += bits # Add the binary string to the bit string.
                byte = input_file.read(1) # Read the next byte.
            
            encoded_text = self.__remove_padding(bit_string)
            decoded_text = self.__decode_text(encoded_text)
            output_file.write(decoded_text)

        # Print the size of the original file and the size of the decompressed file.
        print("The file was decompressed successfully.")
        print(f"\tOriginal file size: {os.path.getsize(input_file_path)} bytes")
        print(f"\tDecompressed file size: {os.path.getsize(output_file_path)} bytes")
        return output_file_path


# Demo

In this demo we compress a small `.txt` file (size = ~5KB). The output file's size is 2.5KB (50% compression rate).

In [2]:
# Demo
huffman = Huffman("data/huffman/20_great_books.txt")
huffman.compress()
huffman.decompress("data/huffman/20_great_books.bin")

The file was compressed successfully.
	Original file size: 4877 bytes
	Compressed file size: 2653 bytes
The file was decompressed successfully.
	Original file size: 2653 bytes
	Decompressed file size: 4876 bytes


'data/huffman/20_great_books_decompressed.txt'

# Observation

### Compression Efficiency
In real life Huffman algorithm results into 50-80% compression ratio. Which is not bad for a lossless compressiong algorithm, discovered 75 years ago. That's why Huffman trees are still being widely used nowadays.

### Other algorithms
There are other popular compression algorithms - like `LZ77` and `LZW`. Latter algorithms however are more appropriate for repetitive data, while Huffman encoding is used for non-repetitive data.

For example, take the string `123123123`.

LZW will identify that `123` is repeated three times and essentially create a dictionary of codes for sequences. It will esentially say when I say `A` I mean `123` here is `AAA` (or 24 bits).

Huffman will detect the frequency of bytes (let's assume the text above is ASCII, which will make `ABC` all single byte code points), so `A=3`, `B=3`, `C=3` and there are no other items, so I can use 1.5 bits (well a 1 and 2 bit combo) to represent all characters. So let's say `A=0`, `B=10`, `C=11`. Huffman will encode the text `ABCABCABC` as (in bits) `010110101101011` (or 15 bits).

### Combination of encoders
What if we used Huffman on the LZW result?
Well `AAA` can be represented with a single bit (Let's choose `0`), so the output would be simply `000` (3 bits only).

# Summary
Huffman trees have been the de-facto standard for optimal encoding since their introduction. Therefore Huffman is widely used in all the mainstream compression formats that you might encounter - from GZIP, PKZIP and BZIP2, to image formats such as JPEG and PNG.

Being familiar with this algorithm and its greedy approach for building prefix trees is a foundamental step in every computer scientist career.